In [1]:
import requests
import requests_cache
from bs4 import BeautifulSoup
import json
import time
import pandas as pd
import datetime as dt
import re
from selenium import webdriver
import urllib.robotparser

In [2]:
rp = urllib.robotparser.RobotFileParser()
rp.set_url('https://tiki.vn/robots.txt')
rp.read()
rp.can_fetch('*', 'product_url = "https://tiki.vn/api/v2/products/{}"')

True

In [3]:
sleep_time = 1
headers = { 'X-Requested-With': 'XMLHttpRequest', 
            "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36"}

In [4]:
laptop_may_vi_tinh_va_linh_kien_url = ''

product_url = "https://tiki.vn/api/v2/products/{}"

In [36]:
def collect_products(urls, products_file):     
    f = open(products_file,'w', encoding = 'utf-8')
    f.write('ID\tName\tDiscount_price\tPrice\tRating\tNum_reviews\tQuantity_sold\n')   
    
    for url in urls:
        i=1
        while (True):
            requests_cache.install_cache()
            r = requests.get(url.format(i), headers=headers)
            if not r.from_cache: 
                time.sleep(sleep_time)
                r = requests.get(url.format(i), headers=headers)

            if (r.status_code != 200):
                break
            items = json.loads(r.text)["data"]

            if (len(items) == 0):
                break

            for item in items:
                f.write(str(item['id']) +'\t')
                f.write(item['name'].replace('\t','')+'\t')
                f.write(str(item['price'])+'\t')
                f.write(str(item['list_price'])+'\t')
                f.write(str(item['rating_average'])+'\t')
                f.write(str(item['review_count'])+'\t')
                if 'quantity_sold' in item:
                    f.write(str(item['quantity_sold']['value'])+'\n')
                else:
                    f.write('0'+'\n')
            i += 1
    f.close() 

In [37]:
urls = ['https://tiki.vn/api/v2/products?limit=48&&category=1846&page={}&urlKey=laptop-may-vi-tinh-linh-kien',
        'https://tiki.vn/api/v2/products?limit=48&&category=1789&page={}&urlKey=dien-thoai-may-tinh-bang',
        'https://tiki.vn/api/v2/products?limit=48&&category=4384&page={}&urlKey=bach-hoa-online']
collect_products(urls, 'products.tsv')

In [38]:
df = pd.read_csv('products.tsv', sep='\t')

In [40]:
df.head(10)

,ID,Name,Discount_price,Price,Rating,Num_reviews,Quantity_sold
0,76181490.0,Phần Mềm Diệt Virus BKAV Profressional 12 Thán...,195000.0,299000.0,4.8,927.0,23907.0
1,416978.0,Bộ Chuyển Đổi USB Wifi Nano TP-Link TL-WN725N ...,117300.0,239000.0,4.5,611.0,6677.0
2,53056161.0,Phần mềm Microsoft 365 Family English APAC EM ...,1190000.0,1690000.0,5.0,720.0,3132.0
3,73795655.0,Apple Macbook Pro 2020 M1 - 13 Inchs (Apple M1...,30990000.0,34990000.0,5.0,209.0,460.0
4,23393172.0,Máy in HP Neverstop Laser 1000w (In/Wifi/Trắn...,3100000.0,4290000.0,0.0,0.0,0.0
5,74975883.0,Bộ Phát Wifi TP-Link Archer C54 Băng Tần Kép C...,459000.0,889000.0,4.8,268.0,655.0
6,54439144.0,Bộ Chuyển Đổi USB Wifi TP-Link Archer T2U Plus...,279000.0,349000.0,4.8,197.0,757.0
7,73795621.0,Apple Macbook Air 2020 M1 - 13 Inchs (Apple M1...,25699000.0,29990000.0,5.0,127.0,253.0
8,35339655.0,Màn Hình Samsung LS24R350FHEXXV 24inch FullHD ...,3575000.0,3990000.0,4.5,119.0,478.0
9,708026.0,Bộ Phát Wifi TP-Link Archer C50 Băng Tần Kép A...,459000.0,999000.0,4.7,127.0,504.0
